In [1]:
%load_ext autoreload
%autoreload 2
import os
import time
import jax

import numpy as np
import pandas as pd
import sys
import yaml

In [2]:
# Install dependencies
!pip install linearmodels

In [3]:
# Mount drive. Make sure everything(including submodules) are there
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
%cd drive/MyDrive/colab_notebooks/policy_unc/analysis

/content/drive/MyDrive/colab_notebooks/policy_unc/analysis


In [ ]:
!git config --global user.email

In [8]:
!git commit -am "colab results"

Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@c78e489f36ec.(none)')


In [5]:
analysis_path = os.getcwd() + "/"

In [6]:
sys.path.insert(0, analysis_path + "submodules/dcegm/src/")
sys.path.insert(0, analysis_path + "src/")
data_decision = pd.read_pickle(analysis_path + "output/decision_data.pkl")
# Retirees don't have any choice and therefore no information
data_decision = data_decision[data_decision["lagged_choice"] != 2]

In [17]:


# Load data specs
from derive_specs import generate_derived_and_data_derived_options

start = time.time()
project_paths = {
    "project_path": analysis_path,
}
project_specs = yaml.safe_load(open(analysis_path + "src/spec.yaml"))
project_specs = generate_derived_and_data_derived_options(
    project_specs, project_paths, load_data=True
)

from model_code.specify_model import specify_model

model, start_params_all, options = specify_model(project_specs, load_model=True)
# Create dummy exog column to handle in the model
data_decision["dummy_exog"] = np.zeros(len(data_decision), dtype=np.int8)
oberved_states_dict = {
    name: data_decision[name].values for name in model["state_space_names"]
}

observed_wealth = data_decision["wealth"].values
observed_choices = data_decision["choice"].values

savings_grid = np.arange(start=0, stop=100, step=0.5)

from dcegm.likelihood import create_individual_likelihood_function_for_model

individual_likelihood = create_individual_likelihood_function_for_model(
    model=model,
    options=options,
    observed_states=oberved_states_dict,
    observed_wealth=observed_wealth,
    observed_choices=observed_choices,
    exog_savings_grid=savings_grid,
    params_all=start_params_all,
)
params_to_estimate_names = ["mu", "delta", "lambda", "sigma"]
start_params = {name: start_params_all[name] for name in params_to_estimate_names}

Estimated regression equation: E[ret age] = 66.80539780412107 + 0.0447874113178399 * (birth year - 1947)
Estimated wage equation coefficients:
                  parameter
constant           0.901966
full_time_exp      0.191656
full_time_exp_sq  -0.002245


In [9]:

past_prep = time.time()
ll = individual_likelihood(start_params)
first = time.time()
print(f"First call took {first - past_prep} seconds.")
ll = individual_likelihood(start_params)
second = time.time()
print(f"Second call took {second - first} seconds.")


KeyboardInterrupt: 

In [ ]:
jax.jacfwd(individual_likelihood)(start_params)

Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x7b384535e5f0>
Traceback (most recent call last):
  File "/usr/lib/python3.10/weakref.py", line 370, in remove
    def remove(k, selfref=ref(self)):
KeyboardInterrupt: 
